    Topic:        Challenge Set 1  
    Subject:      Explore MTA turnstile data  
    Date:         07/09/2018  
    Name:         Courtney  
    Worked with:  Tim, Brandon

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None

**Challenge 1**

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_180630.txt")

### *Pull in another dataset for Challenge 1 *

In [4]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,00:00:00,REGULAR,6667150,2259901
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,04:00:00,REGULAR,6667173,2259909
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,08:00:00,REGULAR,6667189,2259938
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,12:00:00,REGULAR,6667305,2260006
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,16:00:00,REGULAR,6667534,2260066


**Challenge 2**

In [5]:
test_data = data.iloc[0:100]
test_data.shape

(100, 11)

In [6]:
time_series = test_data[['C/A', 'UNIT', 'SCP', 'STATION', 'ENTRIES']]

In [7]:
combined_date = test_data.loc[:,'DATE'] + ' ' + test_data.loc[:, 'TIME']

In [8]:
combined_date = pd.to_datetime(combined_date)

In [9]:
time_series['DATE_TIME'] = combined_date

In [10]:
time_series['DATE'] = pd.to_datetime(test_data.loc[:,'DATE'])

In [11]:
print(time_series.head())

    C/A  UNIT       SCP STATION  ENTRIES           DATE_TIME       DATE
0  A002  R051  02-00-00   59 ST  6667150 2018-06-23 00:00:00 2018-06-23
1  A002  R051  02-00-00   59 ST  6667173 2018-06-23 04:00:00 2018-06-23
2  A002  R051  02-00-00   59 ST  6667189 2018-06-23 08:00:00 2018-06-23
3  A002  R051  02-00-00   59 ST  6667305 2018-06-23 12:00:00 2018-06-23
4  A002  R051  02-00-00   59 ST  6667534 2018-06-23 16:00:00 2018-06-23


In [12]:
time_series.dtypes

C/A                  object
UNIT                 object
SCP                  object
STATION              object
ENTRIES               int64
DATE_TIME    datetime64[ns]
DATE         datetime64[ns]
dtype: object

**Challenge 3**

How many hours apart are the measurements?  
What is the total number of entries per day for each turnstile?  

Subtract a minute from time points to account for 12am registering for the next day.

In [13]:
time_series.loc[:,'DATE_TIME'] = time_series.loc[:,'DATE_TIME'] - pd.Timedelta(minutes=1)

In [14]:
time_series.head()

,C/A,UNIT,SCP,STATION,ENTRIES,DATE_TIME,DATE
0,A002,R051,02-00-00,59 ST,6667150,2018-06-22 23:59:00,2018-06-23
1,A002,R051,02-00-00,59 ST,6667173,2018-06-23 03:59:00,2018-06-23
2,A002,R051,02-00-00,59 ST,6667189,2018-06-23 07:59:00,2018-06-23
3,A002,R051,02-00-00,59 ST,6667305,2018-06-23 11:59:00,2018-06-23
4,A002,R051,02-00-00,59 ST,6667534,2018-06-23 15:59:00,2018-06-23


Create a new Date column with the YYYY-MM-DD format based on the dates after subtracting 1 minute. 

In [15]:
time_series.loc[:,'DATE'] = time_series.loc[:,'DATE_TIME'].dt.date

In [16]:
time_series.head()

,C/A,UNIT,SCP,STATION,ENTRIES,DATE_TIME,DATE
0,A002,R051,02-00-00,59 ST,6667150,2018-06-22 23:59:00,2018-06-22
1,A002,R051,02-00-00,59 ST,6667173,2018-06-23 03:59:00,2018-06-23
2,A002,R051,02-00-00,59 ST,6667189,2018-06-23 07:59:00,2018-06-23
3,A002,R051,02-00-00,59 ST,6667305,2018-06-23 11:59:00,2018-06-23
4,A002,R051,02-00-00,59 ST,6667534,2018-06-23 15:59:00,2018-06-23


Let's see what the time intervals are between measurements. 

In [17]:
group_vals = ['C/A', 'UNIT', 'SCP','STATION']

In [18]:
turnstile_data = time_series.groupby(by=group_vals)

In [19]:
time_diff = turnstile_data['DATE_TIME'].diff()

In [20]:
time_diff.value_counts(normalize=True)

04:00:00    0.979381
08:00:00    0.020619
Name: DATE_TIME, dtype: float64

In the test dataset, the vast majority of time intervals are 4-hour time intervals.   

---
Now let's work on the total entries per turnstile per day. 

In [21]:
entry_diff_per_turnstile_interval = time_series.groupby(by=group_vals)['ENTRIES'].diff()

In [22]:
time_series['entry_per_interval'] = entry_diff_per_turnstile_interval

In [23]:
time_series.iloc[80:90]

,C/A,UNIT,SCP,STATION,ENTRIES,DATE_TIME,DATE,entry_per_interval
80,A002,R051,02-00-01,59 ST,5977278,2018-06-29 15:59:00,2018-06-29,275.0
81,A002,R051,02-00-01,59 ST,5977874,2018-06-29 19:59:00,2018-06-29,596.0
82,A002,R051,02-03-00,59 ST,1093811,2018-06-22 23:59:00,2018-06-22,NaN
83,A002,R051,02-03-00,59 ST,1093819,2018-06-23 03:59:00,2018-06-23,8.0
84,A002,R051,02-03-00,59 ST,1093824,2018-06-23 07:59:00,2018-06-23,5.0
85,A002,R051,02-03-00,59 ST,1093870,2018-06-23 11:59:00,2018-06-23,46.0
86,A002,R051,02-03-00,59 ST,1093973,2018-06-23 15:59:00,2018-06-23,103.0
87,A002,R051,02-03-00,59 ST,1094094,2018-06-23 19:59:00,2018-06-23,121.0
88,A002,R051,02-03-00,59 ST,1094172,2018-06-23 23:59:00,2018-06-23,78.0
89,A002,R051,02-03-00,59 ST,1094181,2018-06-24 03:59:00,2018-06-24,9.0


Remove missing data (dates at the start of each set of turnstile data, where we have no prior date to compare the turnstile count to)

In [24]:
time_series = time_series.dropna(subset=['entry_per_interval'])

In [25]:
grouping = ['C/A', 'UNIT', 'SCP', 'STATION', 'DATE']

In [26]:
daily_entries_per_turnstile = time_series.groupby(by=grouping)['entry_per_interval'].sum().reset_index()

In [27]:
daily_entries_per_turnstile.rename(columns={'entry_per_interval':'DAILY_ENTRIES'}, inplace=True)

In [29]:
daily_entries_per_turnstile.head()

,C/A,UNIT,SCP,STATION,DATE,DAILY_ENTRIES
0,A002,R051,02-00-00,59 ST,2018-06-23,830.0
1,A002,R051,02-00-00,59 ST,2018-06-24,606.0
2,A002,R051,02-00-00,59 ST,2018-06-25,1378.0
3,A002,R051,02-00-00,59 ST,2018-06-26,1234.0
4,A002,R051,02-00-00,59 ST,2018-06-27,1481.0
